In [2]:
import sqlite3
import pandas as pd

# Tạo bảng dữ liệu mẫu
data = [
    ('Smith', 70, 175),
    ('Smith', 70, 180),
    ('Nguyen', 60, 160),
    ('Nguyen', 65, 165),
    ('Tran', 55, 155)
]

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Tạo bảng Patient
cur.execute('''
    CREATE TABLE Patient (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        last_name TEXT,
        weight INTEGER,
        height INTEGER
    )
''')

# Chèn dữ liệu
cur.executemany('INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)', data)
conn.commit()

# Truy vấn: So sánh cặp bệnh nhân và tính khoảng cách Boolean
query = '''
SELECT 
    a.id AS id1,
    b.id AS id2,
    a.last_name, 
    b.last_name,
    a.weight, 
    b.weight,
    CASE WHEN a.last_name = b.last_name THEN 0 ELSE 1 END AS last_name_dist,
    CASE WHEN a.weight = b.weight THEN 0 ELSE 1 END AS weight_dist,
    (CASE WHEN a.last_name = b.last_name THEN 0 ELSE 1 END +
     CASE WHEN a.weight = b.weight THEN 0 ELSE 1 END) AS total_boolean_distance,
    CASE WHEN 
        (CASE WHEN a.last_name = b.last_name THEN 0 ELSE 1 END +
         CASE WHEN a.weight = b.weight THEN 0 ELSE 1 END) = 0 
    THEN 'SAME PERSON'
    ELSE 'DIFFERENT' END AS match_result
FROM Patient a
JOIN Patient b ON a.id < b.id
'''

# Đọc kết quả
df_result = pd.read_sql_query(query, conn)
print(df_result)


   id1  id2 last_name last_name  weight  weight  last_name_dist  weight_dist  \
0    1    2     Smith     Smith      70      70               0            0   
1    1    3     Smith    Nguyen      70      60               1            1   
2    1    4     Smith    Nguyen      70      65               1            1   
3    1    5     Smith      Tran      70      55               1            1   
4    2    3     Smith    Nguyen      70      60               1            1   
5    2    4     Smith    Nguyen      70      65               1            1   
6    2    5     Smith      Tran      70      55               1            1   
7    3    4    Nguyen    Nguyen      60      65               0            1   
8    3    5    Nguyen      Tran      60      55               1            1   
9    4    5    Nguyen      Tran      65      55               1            1   

   total_boolean_distance match_result  
0                       0  SAME PERSON  
1                       2    DIFFEREN

Chỉ duy nhất cặp (id1 = 1, id2 = 2) có:

last_name = "Smith" và "Smith" → giống nhau → last_name_dist = 0

weight = 70 và 70 → giống nhau → weight_dist = 0

total_boolean_distance = 0

→ Kết luận: Chỉ có cặp (1, 2) là có thể là cùng một người.